**Making requests to the OpenAI API**

In [ ]:
!pip install openai

In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key="Paste your OpenAI API key here."
)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": "What is the capital of France?"}
    ]
)

print(response.choices[0].message.content)

In [ ]:
# Accessing API key from the Secrets Vault

from openai import OpenAI
from google.colab import userdata

client = OpenAI(
    api_key=userdata.get('openai_api_key')
    )

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
         {"role": "system", "content": "You are a helpful assistant who provides responses in one-word."},     # Set the tone and persona
         {"role": "user",   "content": "What is the capital of France?"},                                      # Provide the base AI-prompt
    ]
)

print(response)

In [ ]:
print(response.choices[0])

In [ ]:
print(response.choices[0].message)

In [ ]:
print(response.choices[0].message.content)

**Exercise 1. Low complexity use-case: AI-enabled brand extraction**

In [ ]:
import pandas as pd
from openai import OpenAI
from google.colab import userdata

client = OpenAI(
    api_key=userdata.get('openai_api_key')
    )

# Read the raw .csv file
brand_data = pd.read_csv('brand_data.csv')

# Function to extract brand name from description
def extract_brand_name(description):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant who provides responses in a single word."},
            {"role": "user", "content": f"Extract the brand name from the following description {description}"}
        ]
    )
    return response.choices[0].message.content

# Apply the function to the 'Brand Description' column
brand_data['Brand Name'] = brand_data['Brand Description'].apply(extract_brand_name)

brand_data

**Exercise 2. Medium complexity use-case: Tracking character length using loop**

In [ ]:
import pandas as pd
from openai import OpenAI
from google.colab import userdata

client = OpenAI(
    api_key=userdata.get('openai_api_key')
    )

# Read the raw .csv file
brand_data = pd.read_csv('brand_data.csv')

# Function to count characters in brand description
def count_description_characters(description):
  response = client.chat.completions.create(
      model="gpt-4o",
      messages=[
            {"role": "system", "content": "You are a helpful assistant who provides single-word responses by presenting numerical data in a comma-separated format to ensure readability."},
            {"role": "user", "content": f"Count the number of characters in the following description {description}"}
        ]
      )
  return response.choices[0].message.content

# List to hold the brand names
brand_names = []

# Loop through each description in the DataFrame
for description in brand_data['Brand Description']:
    # Count the characters for each description and append to list
    brand_name = count_description_characters(description)
    brand_names.append(brand_name)

# Add the list of brand names as a new column in the DataFrame
brand_data['Character Count'] = brand_names

brand_data

**Exercise 3. High complexity use-case: Analyzing base64 encoded promotions dashboard using OpenAI Vision to generate business report**

In [ ]:
import pandas as pd
from openai import OpenAI
from google.colab import userdata
import base64

client = OpenAI(
    api_key=userdata.get('openai_api_key')
    )

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


# Path to your image
image_path = "/content/retail_dashboard_promos.jpg"

# Getting the Base64 string
base64_image = encode_image(image_path)

system_prompt = """

Act as a business analyst having specialization in the Retail domain.

"""

user_prompt = """

Prompt: Analyze the attached promotional analysis dashboard image and provide a detailed report.
Explain the performance trends, identify key insights on product sales and promotions,
and suggest potential business actions based on the data observed.

Additional Context:
- The main focus is to understand how promotions impact sales volume and stock availability.
- Structure the results properly separating each section to improve readability.
- Tabulate information wherever applicable.

"""

response = client.chat.completions.create(
    model="o1",
    messages=[
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": user_prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                },
            ],
        }
    ],
)

print(response.choices[0].message.content)

**AI Model: o3-mini (Property: Function calling)**

In [ ]:
import json  # Needed to parse the JSON string from the API
from openai import OpenAI
from google.colab import userdata

# Initialize OpenAI client
client = OpenAI(api_key=userdata.get('openai_api_key'))

# Define the function that we want the model to call
def operator_func(a: int, b: int) -> int:
    return a**2 + b**2  # Function logic (squaring and adding two numbers)

# Define function calling
functions = [
    {
        "name": "operator_func",
        "description": "Calculates the sum of squares of two numbers.",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {"type": "integer", "description": "First number"},
                "b": {"type": "integer", "description": "Second number"},
            },
            "required": ["a", "b"],
        },
    }
]

# Send request with function calling enabled
response = client.chat.completions.create(
    model="o3-mini",
    messages=[{"role": "user", "content": "What is the result when a=2 and b=4 are fed into operator_func()?"}],
    functions=functions,
    function_call="auto"  # Allow model to call functions
)

# Extract function call arguments
if response.choices[0].message.function_call:
    function_name = response.choices[0].message.function_call.name
    arguments_json = response.choices[0].message.function_call.arguments  # This is a string

    # Convert string to dictionary
    arguments = json.loads(arguments_json)

    if function_name == "operator_func":
        result = operator_func(**arguments)  # Arguments is a dictionary
        print(f"Function Output: {result}")
    else:
        print("Unexpected function call")
else:
    print("No function call detected.")

Function Output: 20


**AI Model: o3-mini (Property: Structured output)**

In [ ]:
import json  # Needed to parse the JSON string from the API
from openai import OpenAI
from google.colab import userdata

# Initialize OpenAI client
client = OpenAI(api_key=userdata.get('openai_api_key'))

# Define the desired output structure with 'additionalProperties': false
response_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "math_solution",
        "schema": {
            "type": "object",
            "properties": {
                "steps": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "explanation": {"type": "string"},
                            "equation": {"type": "string"},
                        },
                        "required": ["explanation", "equation"],
                        "additionalProperties": False  # Ensures no extra properties
                    },
                },
                "final_answer": {"type": "string"},
            },
            "required": ["steps", "final_answer"],
            "additionalProperties": False  # Required to prevent unknown properties
        },
        "strict": True,
    },
}

# Ask the model to solve a math problem
response = client.chat.completions.create(
    model="o3-mini",
    messages=[{"role": "user", "content": "Solve 2x + 3 = 7"}],
    response_format=response_format
)

print(response.choices[0].message.content)

{
  "steps": [
    {
      "explanation": "Subtract 3 from both sides.",
      "equation": "2x = 7 - 3"
    },
    {
      "explanation": "Simplify the right side.",
      "equation": "2x = 4"
    },
    {
      "explanation": "Divide both sides by 2.",
      "equation": "x = 2"
    }
  ],
  "final_answer": "x = 2"
}


**AI Model: o3-mini (Property: Developer messages)**

In [ ]:
import json  # Needed to parse the JSON string from the API
from openai import OpenAI
from google.colab import userdata

# Initialize OpenAI client
client = OpenAI(api_key=userdata.get('openai_api_key'))

# Developer message: Ensure AI asks for required data before suggesting pricing
developer_message = {
    "role": "developer",
    "content": (
        "Before providing a pricing recommendation, ALWAYS ask for the following details if they are missing:"
        "\n1. Product category (e.g., electronics, apparel, groceries)"
        "\n2. Current inventory levels"
        "\n3. Competitor pricing (if available)"
        "\nIf the user doesn’t provide these, ask follow-up questions first before making a suggestion."
    )
}

# Send request with function calling enabled
response = client.chat.completions.create(
    model="o3-mini",
    messages=[
        developer_message,
        {"role": "user", "content": "Should I discount my products this month?"}
    ],
)

print(response.choices[0].message.content)

Before providing a pricing recommendation, could you please provide a few additional details?

1. What is the product category (e.g., electronics, apparel, groceries)?
2. What are your current inventory levels?
3. Do you have any information on competitor pricing?

These details will help me offer a more tailored recommendation for your situation.
